In [ ]:
from rdkit.Chem import Descriptors, AllChem as Chem, DataStructs, PandasTools
import numpy as np
import pickle
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [ ]:
def calc_descriptors(rdmol):
    try:
        logp = Descriptors.MolLogP(rdmol)
        mwt = Descriptors.MolWt(rdmol)
        rtb = Descriptors.NumRotatableBonds(rdmol)
        hbd = Descriptors.NumHDonors(rdmol)
        hba = Descriptors.NumHAcceptors(rdmol)
        tpsa = Descriptors.TPSA(rdmol)
        return [logp, mwt, rtb, hbd, hba, tpsa]
    except:
        return [999, 999, 999, 999, 999, 999, 999]
    
def calc_fingerprint(rdmol, N_BITS=1024):
        fp = Chem.GetMorganFingerprintAsBitVect(
            rdmol, radius=2, nBits=N_BITS, useFeatures=False
        )
        np_fp = np.zeros(0)
        ecfp = DataStructs.ConvertToNumpyArray(fp, np_fp)
        return np_fp

def pred_category(p0, p1, significance):
    if (p0 >= significance) & (p1 >= significance):
        return "both"
    if (p0 >= significance) & (p1 < significance):
        return "inactive"
    if (p0 < significance) & (p1 >= significance):
        return "active"
    else:
        return "empty"

In [ ]:
df = pd.read_csv('herg2.csv', sep=',',names=['smiles', 'ic50'])
PandasTools.AddMoleculeColumnToFrame(df, 'smiles', 'mol')
df['mol'] = df['mol'].fillna(0)
df = df[df.mol != 0]

In [ ]:
df['active'] = [1 if v > 6 else 0 for v in df.ic50]

In [ ]:
df['active'].plot(kind='hist')

In [ ]:
features = [np.append(calc_descriptors(mol), calc_fingerprint(mol)) for mol in df.mol]

In [ ]:
import os
import pickle
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

scaler = preprocessing.StandardScaler().fit(features)

if not os.path.exists('models'):
    os.makedirs('models')
pickle.dump(scaler, open('models/scaler-herg.pkl','wb'))

X= scaler.transform(features)
y = df['active'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
lgb_train = lgb.Dataset(X_train, y_train)  
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)  

In [ ]:
params = {  
    'boosting_type': 'gbdt',  
    'objective': 'binary',  
    'metric': {'binary_logloss', 'auc'},  #二进制对数损失
    'num_leaves': 5,  
    'max_depth': 6,  
    'min_data_in_leaf': 450,  
    'learning_rate': 0.1,  
    'feature_fraction': 0.9,  
    'bagging_fraction': 0.95,  
    'bagging_freq': 5,  
    'lambda_l1': 1,    
    'lambda_l2': 0.001,  # 越小l2正则程度越高  
    'min_gain_to_split': 0.2,  
    'verbose': 5,  
    'is_unbalance': True  
}
# train  
print('Start training...')  
gbm = lgb.train(params,  
                lgb_train,  
                num_boost_round=10000,  
                valid_sets=lgb_eval,  
                early_stopping_rounds=500)  

print('Start predicting...')  
preds = gbm.predict(X_test, num_iteration=gbm.best_iteration)
auc_lgbm = roc_auc_score(y_test,preds)
print(auc_lgbm)

In [ ]:
if not os.path.exists('models'):
    os.makedirs('models')
pickle.dump(gbm, open('models/gbm-classifier-herg.pkl','wb'))

In [ ]:
gbm = pickle.load(open('models/gbm-classifier-herg.pkl', 'rb'))

In [ ]:
y_pred = gbm.predict(X_test)
print('The AUC of prediction is:', roc_auc_score(y_test,preds))

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

fpr = dict()
tpr = dict()
roc_auc = dict()
fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC of hERG')
plt.legend(loc="lower right")
plt.savefig('herg_predict.svg')
plt.show()

# Load models and predict molecules (smiles)

In [ ]:
import pandas as pd
import pickle
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem as Chem, DataStructs
from rdkit.Chem import PandasTools
import numpy as np

def calc_descriptors(rdmol):
    try:
        logp = Descriptors.MolLogP(rdmol)
        mwt = Descriptors.MolWt(rdmol)
        rtb = Descriptors.NumRotatableBonds(rdmol)
        hbd = Descriptors.NumHDonors(rdmol)
        hba = Descriptors.NumHAcceptors(rdmol)
        tpsa = Descriptors.TPSA(rdmol)
        return [logp, mwt, rtb, hbd, hba, tpsa]
    except:
        return [999, 999, 999, 999, 999, 999, 999]
    
def calc_fingerprint(rdmol, N_BITS=1024):
        fp = Chem.GetMorganFingerprintAsBitVect(
            rdmol, radius=2, nBits=N_BITS, useFeatures=False
        )
        np_fp = np.zeros(0)
        ecfp = DataStructs.ConvertToNumpyArray(fp, np_fp)
        return np_fp

def pred_category(p0, p1, significance):
    if (p0 >= significance) & (p1 >= significance):
        return "both"
    if (p0 >= significance) & (p1 < significance):
        return "inactive"
    if (p0 < significance) & (p1 >= significance):
        return "active"
    else:
        return "empty"


df = pd.read_csv('smiles_list.smi', sep='\t',names=['smiles', 'cas', 'active'])
PandasTools.AddMoleculeColumnToFrame(df, 'smiles', 'mol')
df['mol'] = df['mol'].fillna(0)
df = df[df.mol != 0]
features = [np.append(calc_descriptors(mol), calc_fingerprint(mol)) for mol in df.mol.values]
scaler = pickle.load(open('models/scaler-herg.pkl', 'rb'))
X = scaler.transform(features)
gbm = pickle.load(open('models/gbm-classifier-herg.pkl', 'rb'))
y_pred = gbm.predict(X)

In [ ]:
import pandas as pd
import pickle
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem as Chem, DataStructs
from rdkit.Chem import PandasTools

def calc_descriptors(rdmol):
    try:
        logp = Descriptors.MolLogP(rdmol)
        mwt = Descriptors.MolWt(rdmol)
        rtb = Descriptors.NumRotatableBonds(rdmol)
        hbd = Descriptors.NumHDonors(rdmol)
        hba = Descriptors.NumHAcceptors(rdmol)
        tpsa = Descriptors.TPSA(rdmol)
        return [logp, mwt, rtb, hbd, hba, tpsa]
    except:
        return [999, 999, 999, 999, 999, 999, 999]
    
def calc_fingerprint(rdmol, N_BITS=1024):
        fp = Chem.GetMorganFingerprintAsBitVect(
            rdmol, radius=2, nBits=N_BITS, useFeatures=False
        )
        np_fp = np.zeros(0)
        ecfp = DataStructs.ConvertToNumpyArray(fp, np_fp)
        return np_fp


def herg_score(mol):
    try:
        smi = Chem.MolToSmiles(mol)
        features = np.append(calc_descriptors(mol), calc_fingerprint(mol))
        features = features.reshape(1, -1)
        scaler = pickle.load(open('models/scaler-herg.pkl', 'rb'))
        X = scaler.transform(features)
        gbm = pickle.load(open('models/gbm-classifier-herg.pkl', 'rb'))
        y_pred = gbm.predict(X)
        return y_pred[0]
    except Exception as e:
        return print(e)

In [ ]:
smi = 'CCN(CC)CCCC(C)NC1=C2C=CC(=CC2=NC=C1)Cl'
mol = Chem.MolFromSmiles(smi)
Chem.Kekulize(mol)
herg_score(mol)